# Лабораторна робота №3: "Наука про дані: обмін результатами та початковий аналіз" 
### *виконала студентка групи ФБ-33 Журавльова Марія*

**Мета роботи:** ознайомитися з системою контролю версій GitHub, навчитися створювати прості веб-додатки для обміну результатами досліджень із використанням модуля Streamlit.

Підготую дані:

In [18]:
from download_csv import *

country = "UKR"
year_1 = 1981
year_2 = 2024
type_data = "Mean"
dir_data = r"C:/Users/User/vhi_venv/data_csv"

clean_directory(dir_data)
download_csv(country, year_1, year_2, type_data, dir_data)
df = load_vhi_data(dir_data)
df_replace = replace_region_ids(df)

print("Починається збереження у all_vhi.csv...")
df_replace.to_csv("all_vhi.csv", index=False)
print("Дані збережено успішно!")

Директорія очищена від старих файлів.
Файл VHI_province_1_20250326154845.csv завантажено успішно.
Файл VHI_province_2_20250326154846.csv завантажено успішно.
Файл VHI_province_3_20250326154847.csv завантажено успішно.
Файл VHI_province_4_20250326154848.csv завантажено успішно.
Файл VHI_province_5_20250326154849.csv завантажено успішно.
Файл VHI_province_6_20250326154851.csv завантажено успішно.
Файл VHI_province_7_20250326154853.csv завантажено успішно.
Файл VHI_province_8_20250326154854.csv завантажено успішно.
Файл VHI_province_9_20250326154855.csv завантажено успішно.
Файл VHI_province_10_20250326154856.csv завантажено успішно.
Файл VHI_province_11_20250326154856.csv завантажено успішно.
Файл VHI_province_12_20250326154857.csv завантажено успішно.
Файл VHI_province_13_20250326154858.csv завантажено успішно.
Файл VHI_province_14_20250326154859.csv завантажено успішно.
Файл VHI_province_15_20250326154900.csv завантажено успішно.
Файл VHI_province_16_20250326154901.csv завантажено успі

**Хід виконання роботи:**

Створення веб-додатоку із використанням модуля Streamlit:
1. Створіть dropdown список, який дозволить обрати часовий ряд VCI, TCI, VHI для набору даних із лабораторної роботи 2;
2. Створіть dropdown список, який дозволить вибрати область, для якої буде виконуватись аналіз;
3. Створіть slider, який дозволить зазначити інтервал тижнів, за які відбираються дані;
4. Створіть slider, який дозволить зазначити інтервал років, за які відбираються дані;
5. Створіть button для скидання всіх фільтрів і повернення до початкового стану даних (відповідно інтерактивні елементи повинні мати початкові значення);
6. Створіть три вкладки для відображення таблиці з відфільтрованими даними, відповідного до неї графіка та графіка порівняння даних по областях.
7. Перший графік повинен відображати відфільтровані дані (часові ряди за діапазон років, що обмежені інтервалом тижнів). Другий графік має відображати порівняння значень VCI, TCI або VHI (залежно від обраної опції у списку dropdown) для обраної області з усіма іншими областями за вказаний часовий інтервал. Продумайте вигляд цих графіків.
8. Створіть два checkbox для сортування даних за зростанням та спаданням значень VCI, TCI або VHI (залежно від обраної опції у списку dropdown). Продумайте реакцію програми, якщо увімкнені обидва чекбокси.
9. Інтерактивні елементи мають бути розміщений в одній колонці, а графіки з таблицею — в іншій.

In [ ]:
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Завантаження даних з CSV (кешоване)
@st.cache_data
def load_from_file():
    return pd.read_csv("all_vhi.csv")

df_replace = load_from_file()

area_names = {
    1: 'Вінницька', 2: 'Волинська', 3: 'Дніпропетровська', 4: 'Донецька', 5: 'Житомирська',
    6: 'Закарпатська', 7: 'Запорізька', 8: 'Івано-Франківська', 9: 'Київська', 10: 'Кіровоградська',
    11: 'Луганська', 12: 'Львівська', 13: 'Миколаївська', 14: 'Одеська', 15: 'Полтавська',
    16: 'Рівенська', 17: 'Сумська', 18: 'Тернопільська', 19: 'Харківська', 20: 'Херсонська',
    21: 'Хмельницька', 22: 'Черкаська', 23: 'Чернівецька', 24: 'Чернігівська', 25: 'Республіка Крим'
}

# Назва додатку
st.title("Лабораторна №3")

# Параметри діапазонів
index_options = ["VCI", "TCI", "VHI"]
min_year, max_year = int(df_replace["Year"].min()), int(df_replace["Year"].max())
min_week, max_week = int(df_replace["Week"].min()), int(df_replace["Week"].max())

if "index" not in st.session_state:
    st.session_state["index"] = "VCI"
if "area" not in st.session_state:
    st.session_state["area"] = list(area_names.values())[0]
if "weeks" not in st.session_state:
    st.session_state["weeks"] = (min_week, max_week)
if "years" not in st.session_state:
    st.session_state["years"] = (min_year, max_year)
if "sort_asc" not in st.session_state:
    st.session_state["sort_asc"] = False
if "sort_desc" not in st.session_state:
    st.session_state["sort_desc"] = False

# 9 - 2 колонки: фільтри та результат
col1, col2 = st.columns([1, 2])

# Колонка з фільтрами (інтерактивними елементами)
with col1:
    st.subheader("Фільтри")

    # 1 - dropdown список, який дозволить обрати часовий ряд VCI, TCI, VHI
    selected_index = st.selectbox("Оберіть часовий ряд", index_options, key="index")

    # 2 - dropdown список, який дозволить вибрати область
    selected_area = st.selectbox("Оберіть область", list(area_names.values()), index=0, key="area")
    area_id = list(area_names.keys())[list(area_names.values()).index(selected_area)]

    # 3 - slider, який дозволить зазначити інтервал тижнів
    week_range = st.slider("Інтервал тижнів", min_week, max_week, key="weeks")

    # 4 - slider, який дозволить зазначити інтервал років
    year_range = st.slider("Інтервал років", min_year, max_year, key="years")

    # 8 - два checkbox для сортування даних за зростанням та спаданням значень VCI, TCI або VHI
    asc = st.checkbox("Сортувати за зростанням", key="sort_asc")
    desc = st.checkbox("Сортувати за спаданням", key="sort_desc")

    # 5 - button для скидання всіх фільтрів і повернення до початкового стану даних
    if st.button("Скинути фільтри"):
        st.session_state.clear()
        st.rerun()

# Фільтрація даних за вибраними параметрами
filtered_df = df_replace[
    (df_replace["Region_ID"] == area_id) &
    (df_replace["Year"].between(year_range[0], year_range[1])) &
    (df_replace["Week"].between(week_range[0], week_range[1]))
].copy()

# 8 - Сортування
if asc and not desc:
    filtered_df = filtered_df.sort_values(by=selected_index, ascending=True)
elif desc and not asc:
    filtered_df = filtered_df.sort_values(by=selected_index, ascending=False)
elif asc and desc:
    st.warning("!!!! Увімкнені обидва сортування — сортування не застосовано.")

# Колонка з графіками та таблицею
with col2:
    # 6 - три вкладки для відображення таблиці з відфільтрованими даними, відповідного до неї графіка та графіка порівняння даних по областях
    tab1, tab2, tab3 = st.tabs(["Таблиця", "Графік області", "Порівняння по областях"])

    with tab1:
        st.write("Відфільтровані дані:")
        filtered_df["Область"] = filtered_df["Region_ID"].map(area_names)
        st.dataframe(filtered_df[["Область", "Year", "Week", selected_index]])

with tab2:
    st.write("Графік індексу для обраної області")
    if not filtered_df.empty:
        fig, ax = plt.subplots(figsize=(10, 5))
        sns.lineplot(data=filtered_df, x="Week", y=selected_index, hue="Year", ax=ax)
        ax.set_title(f"{selected_index} для {selected_area}")
        ax.set_ylabel(selected_index)
        ax.set_xlabel("Week")
        st.pyplot(fig)
    else:
        st.warning("Немає даних для побудови графіка.")

    with tab3:
        st.write("Порівняння по регіонах")
        compare_df = df_replace[
            (df_replace["Year"].between(year_range[0], year_range[1])) &
            (df_replace["Week"].between(week_range[0], week_range[1]))
        ]
        avg_values = compare_df.groupby("Region_ID")[selected_index].mean().reset_index()
        avg_values["Область"] = avg_values["Region_ID"].map(area_names)

        fig2, ax2 = plt.subplots(figsize=(14, 7))
        sns.barplot(data=avg_values, x="Область", y=selected_index, ax=ax2)
        ax2.set_title(f"Середнє значення {selected_index} по регіонах")
        ax2.set_ylabel(f"{selected_index}")
        ax2.set_xlabel("Область")
        ax2.tick_params(axis='x', rotation=45)
        st.pyplot(fig2)

Цей код записую у файл Lab.py і перевіряю:

In [9]:
!streamlit run Lab3.py

^C


**Робота з GitHub:**
1. Зареєструйтесь на GitHub;
2. Створіть public репозиторій;
3. Додайте другу та третю лабораторні роботи в цей репозиторій. Кожна лабораторна
повинна бути в окремій директорії;
4. Репозиторій повинен мати заповнений файл README.MD та .gitignore, в якому
прописані винятки для Git: таблиці та інші дані, що використовуються.

Створюю акаунт *teddy-mary* та репозиторій *data-analisis*. Додаю 2 лабораторну:

```bash
git clone https://github.com/teddy-mary/data-analisis.git
cd data-analisis
mkdir lab2 lab3
copy "C:\Users\User\OneDrive\Desktop\university\ЗАПД\Lab2_FB-33_Zhuravleva.ipynb" lab2\
git add .
git config --global user.name "teddy-mary"
git config --global user.email "marizhur-ipt28@lll.kpi.ua"
git commit -m "Lab2"
git push origin main
```
Аналогічно додаю файли і по 3 лабораторній.